# Analysis of fringes taken on October 27 2020
## Louise + JC
### Inspired from Louise "fringes_analaysis.py"

This is an old code see the two nore recent ones called:
* Generate-Fringes-Oct-2020.Rmd for produciton of the fringes
* Analayse- Fringes-Oct-2020.Rmd for analysis of the fringes

In [ ]:
from pylab import *
import os
import sys
import time
import pickle
from importlib import reload


# Specific science modules
import healpy as hp
import scipy
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sop
import pandas as pd

from matplotlib.backends.backend_pdf import PdfPages
import qubic
from qubic import selfcal_lib as sc
from qubicpack.utilities import Qubic_DataDir
from qubicpack import qubicpack as qp
from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
from qubic import fringes_lib as fl

In [ ]:
# ============== Get data ==============
# global_dir = '/home/lmousset/QUBIC/Qubic_work/Calibration/datas/'
global_dir = '/Users/hamilton/Qubic/Calib-TD/Fringes/'

datasets = np.sort(glob.glob(global_dir+'/*RF_switch*'))

horns = []
ncycles = []
wt = []
for ds in datasets:
    strs = ds.split('_')
    ncycles.append(float(strs[strs.index('ncycles')+1]))
    wt.append(float(strs[strs.index('wt')+1]))
    horns.append([int(strs[-1]), int(strs[-2])])

print('Ncycles')
print(ncycles)
print('WT')
print(wt)
print('Horns')
print(horns)

In [ ]:
ids = 0
mywt = wt[ids]
mync = ncycles[ids]
tes1 = 94
tes2 = 38
doplot = True
_, t_data1, data1 = fl.get_data(datasets, ids, 1, doplot=doplot, tes=tes1)
figure()
_, t_data2, data2 = fl.get_data(datasets, ids, 2, doplot=doplot, tes=tes2)

In [ ]:
asic = 1
if asic==1:
    t_data = t_data1
    data = data1
    tes = tes1
else:
    t_data = t_data2
    data = data2
    tes = tes2
    

# Find the right period
ppp, rms, period = fl.find_right_period(6*mywt/1000, t_data, data[tes - 1, :])
print('period : ', ppp[np.argmax(rms)])
print('Expected : ', 6*mywt/1000)

plt.subplot(211)
plt.plot(ppp, rms, '.')
plt.axvline(x=period, color='orange')


myperiod = period.copy()
plt.subplot(212)
plt.plot(t_data % (myperiod), data[tes - 1, :], '.')
plt.xlim(0, myperiod)

In [ ]:
# Filter the data (just to give an idea because it is done when folding)
lowcut = 0.00001
highcut = 4.
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])

newdata = ft.filter_data(t_data, data[tes-1, :], lowcut, highcut, notch=notch,
                         rebin=True, verbose=True, order=5)

spectrum_f, freq_f = ft.power_spectrum(t_data, data[tes-1, :], rebin=True)
spectrum_f2, freq_f2 = ft.power_spectrum(t_data, newdata, rebin=True)

rc('figure', figsize=(16, 10))
plt.figure()
plt.subplot(211)
plt.plot(freq_f, spectrum_f, label='Original')
plt.plot(freq_f2, spectrum_f2, label='filtered')
plt.legend()
plt.loglog()
plt.ylim(1e0, 1e13)

plt.subplot(212)
plt.plot(t_data, data[tes-1, :], label='Original')
plt.plot(t_data, newdata, label='Filtered')
plt.legend()
plt.show()

In [ ]:
reload(ft)
reload(fl)
# Fold and filter the data
nbins = 120
folded, t, folded_nonorm, newdata = ft.fold_data(t_data,
                                                 data,
                                                 period,
                                                 lowcut,
                                                 highcut,
                                                 nbins,
                                                 notch=notch,
                                                 median=False,
                                                )


# ========== Fit folded signal ================
param_guess = [0.1, 0., 1, 1, 1, 1, 1, 1]
stable_time = period/6
fit = sop.least_squares(fl.make_diff_sig,
                        param_guess,
                        args=(t,
                              stable_time,
                              folded[tes1 - 1, :]),
                        bounds=([0., -2, -2, -2, -2, -2, -2, -2],
                                [1., 2, 2, 2, 2, 2, 2, 2]),
                        verbose=1
                        )
param_est = fit.x
print('Param_est :', param_est)
x0_est = param_est[1]
amps_est = param_est[2:8]
residuals = folded[tes - 1, :]-ft.simsig_fringes(t, stable_time, param_est)
rms_residuals = np.std(residuals)

#### Make the combbination
comb = fl.make_combination(param_est)
print(comb)
#===============================================


plt.figure()
plt.subplot(211)
plt.plot(t_data % period, data[tes - 1, :],',', alpha=0.1)
plt.title('Data cut')
plt.xlim(0, period)

plt.subplot(212)
plt.plot(t, folded[tes - 1, :],label='Average')
plt.plot(t, ft.simsig_fringes(t, stable_time, param_est), label='fit')
plt.plot(np.arange(0, 6 * stable_time, stable_time) + x0_est, amps_est, 'o', label='amplitudes')
plt.title('ASIC {}, TES {}'.format(asic, tes1))
plt.plot(t, residuals, 
         label='Residuals '+' {0:5.3f}'.format(rms_residuals))

plt.title('Folded data')
plt.xlim(0, period)
plt.legend()
plt.show()

In [ ]:
# ========= Michel's method ===================
# w is made to make the combination to see fringes
tm1 = 12
tm2 = 2
ph = 5
w = np.zeros_like(t)
wcheck = np.zeros_like(t)
print(len(w))
per = len(w) / 6
for i in range(len(w)):
    if (((i - ph) % per) >= tm1) and (((i - ph) % per) < per - tm2):
        if (((i - ph) // per) == 0) | (((i - ph) // per) == 3):
            w[i] = 1.
        if (((i - ph) // per) == 1) | (((i - ph) // per) == 2):
            w[i] = -1.

npts = np.sum(w != 0.) / 4.

print(npts)
print(np.sum(np.abs(w[int(per + ph):int(2 * per + ph)])))
print(np.sum(w))

themax = np.max(folded[tes - 1, :])

plt.figure()
plt.plot(t, folded[tes - 1, :])
plt.plot(t, w * themax, 'o')
plt.plot(t, wcheck * themax, 'x')
plt.xlim(0, period)
plt.grid()
plt.show()

In [ ]:
# Get a dictionary
basedir = Qubic_DataDir(datafile='instrument.py', )
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)

rc('figure', figsize=(16,3))
def plot_horns(q):
    hcenters = q.horn.center[:,0:2]
    fig1, ax1 = plt.subplots()
    ax1.set_aspect('equal')
    plot(hcenters[:,0], hcenters[:,1], 'ro')
#     for i in range(64):
#         text(hcenters[i,0], hcenters[i,1], i+1)
    
def plot_baseline(q,bs):
    hcenters = q.horn.center[:,0:2]
    plot(hcenters[np.array(bs)-1,0], hcenters[np.array(bs)-1,1], lw=4, label=bs)
    
plot_horns(q)
plot_baseline(q, [1,2])
legend()

# Now the same looping over TES and Datasets

In [ ]:
# ============== Get data ==============

#### 27/10/2020
#### Vtes = 5
global_dir = '/Volumes/LaCie/QubicData/Calib-TD/Fringes/2020-10-27/'
datasets = np.sort(glob.glob(global_dir+'/*RF_switch*'))
equiv = [0,0,1,1,2,2,3,3,3,3,3,3,3,3]    

#### 28/10/2020
# global_dir = '/Volumes/LaCie/QubicData/Calib-TD/Fringes/2020-10-28/'
#### Vtes=4
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_Vtes_4_*'))
# equiv = [0,0,1,1,2,2,3,3]    
#### Vtes=3.5
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_Vtes_3.5_*'))
# equiv = [0,0,1,1,2,2,3,3]    
#### Vtes=4 & No Eccosorb
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_NoEco_Vtes_4_*'))
# equiv = [0,0,1,1,2,2,3,3]    


horns = []
ncycles = []
wt = []
for ds in datasets:
    strs = ds.split('_')
    ncycles.append(float(strs[strs.index('ncycles')+1]))
    wt.append(float(strs[strs.index('wt')+1]))
    horns.append([int(strs[-2]), int(strs[-1])])

print('Ncycles')
print(ncycles)
print('WT')
print(wt)
print('Horns')
print(horns)

print()
all_equiv = np.unique(equiv)

print('equivalency of baselines')
print(equiv)
print()
print()
print()
print('Found {} equivalency classes:'.format(len(all_equiv)))
for i in range(len(all_equiv)):
    dsequiv = where(np.array(equiv)==all_equiv[i])[0]
    print(' - Type {}'.format(all_equiv[i]))
    for j in range(len(dsequiv)):
        print('     * dsnum={} [{},{}]'.format(dsequiv[j], horns[dsequiv[j]][0], horns[dsequiv[j]][1]))
   


In [ ]:
for i in range(len(all_equiv)):
    dsequiv = where(np.array(equiv)==all_equiv[i])[0]
    figure()
    plot_horns(q)
    print(' - Type {}'.format(all_equiv[i]))
    for j in range(len(dsequiv)):
        print('     * dsnum={} [{},{}]'.format(dsequiv[j], horns[dsequiv[j]][0], horns[dsequiv[j]][1]))
        plot_baseline(q,[horns[dsequiv[j]][0], horns[dsequiv[j]][1]])
    legend(fontsize=5)

In [ ]:
plot_horns(q)
plot_baseline(q,[2,6])
plot_baseline(q,[10,14])
legend(fontsize=5)


In [ ]:
# ============ Analysis for both ASICs and all measurements ==================
reload(fl)

myequiv = 0
mydsequiv = where(np.array(equiv)==myequiv)[0]
print(mydsequiv)


#mydsequiv = [mydsequiv[1], mydsequiv[5]]

print('=============================================================================================')
print('Type {}'.format(myequiv))
for j in range(len(mydsequiv)):
        print('     * dsnum={} [{},{}]'.format(mydsequiv[j], horns[mydsequiv[j]][0], horns[mydsequiv[j]][1]))
print('=============================================================================================')


myw = None
stable_time = period/6

all_folded = []
all_params = []
all_res_louise = []

for ids in mydsequiv:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++ Doing: "+datasets[ids])
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    t, folded_bothasics, param_est, res_mich, res_louise = fl.analyse_fringes(datasets, ids, myw, 
                                                                        lowcut=lowcut, highcut=highcut, notch=notch,
                                                                        t0=None, tf=None, 
                                                                        stable_time=period/6)
    all_folded.append(folded_bothasics)
    all_params.append(param_est)
    all_res_louise.append(res_louise)


In [ ]:
# =================== Make a mask ==============
# Mask to remove the 8 thermometer pixels
mask = np.ones((17,17))
mask[0, 12:] = np.nan
mask[1:5, 16] = np.nan

def qgrid():
    for i in range(17):
        axvline(x=i-0.5, alpha=0.3, color='k')
        axhline(y=i-0.5, alpha=0.3, color='k')


neq = len(mydsequiv)

tes = 95

ny=1
if neq>4:
    ny = 2
nx = neq/ny

all_fringes = []
all_pairs = []

rc('figure', figsize=(16,7))

for i in range(len(mydsequiv)):
    ids = mydsequiv[i]
    param_est = all_params[i]
    folded_bothasics = all_folded[i]
    res_louise = all_res_louise[i]
    baselines = "[{},{}]".format(horns[ids][0], horns[ids][1])
    
    # ============== Plots =============
    # Look at one fit
    x0_est = param_est[tes - 1, 1]
    amps_est = param_est[tes - 1, 2:8]
    
    subplot(ny,nx,i+1)
    plt.plot(t, folded_bothasics[tes - 1, :], label='folded signal')
    plt.plot(t, ft.simsig_fringes(t, stable_time, param_est[tes - 1, :]), label='fit')
    plt.plot(np.arange(0, 6 * stable_time, stable_time) + x0_est, amps_est, 'ro', label='amplitudes')
    resid = folded_bothasics[tes - 1, :]-ft.simsig_fringes(t, stable_time, param_est[tes - 1, :])
    plt.plot(t, resid, label='Residuals: RMS={0:6.4f}'.format(np.std(resid)))
    name = datasets[mydsequiv[i]].split('/')[-1].split('_')
    title('Baseline {}'.format(horns[ids])+'\n'+name[0]+' '+name[1])
    legend(loc='upper right')
    ylim(-1,2.5)
    
    fringe_louise = ft.image_asics(all1=res_louise)
    fringe_louise /= np.nanstd(fringe_louise)

    all_fringes.append(fringe_louise)
    all_pairs.append(horns[ids])

all_names = ''
figure()
for i in range(len(all_fringes)):
    subplot(ny,nx,i+1)
    lim = 2
    plt.imshow(nan_to_num(all_fringes[i] * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
    qgrid()
    name = datasets[mydsequiv[i]].split('/')[-1].split('_')
    myname = name[0]+' '+name[1]+' ; '
    all_names = all_names + myname
    plt.title('{} - Method: Louise'.format(all_pairs[i])+'\n'+myname)
    plt.colorbar()

sgns = np.ones_like(np.array(all_fringes))
for i in range(neq):
    sgns[i,:,:] *= (-1)**i

tight_layout()

In [ ]:
rc('figure', figsize=(16,7))
sgns = np.ones_like(np.array(all_fringes))
for i in range(neq):
    sgns[i,:,:] *= (-1)**i

av_fringe = np.sum(np.array(all_fringes), axis=0)/2
diff_fringe = np.sum(np.array(all_fringes)*sgns, axis=0)/2
lim = 2
subplot(1,2,1)
plt.imshow(nan_to_num(av_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
qgrid()
plt.title('Sum / {}'.format(neq)+'\n'+all_names)
plt.colorbar()
subplot(1,2,2)
plt.imshow(nan_to_num(diff_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
qgrid()
plt.title('diff / {}'.format(neq)+'\n'+all_names)
plt.colorbar()
tight_layout()

# Theoretical Fringes (for fitting)

In [ ]:
import qubic.selfcal_lib as sc
# Use a tool from qubicpack to get a path
basedir = Qubic_DataDir(datafile='instrument.py', )
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'

# Get a dictionary and an instrument
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)


In [ ]:
reload(sc)

# Create an object
baseline = [49, 53]
ca = sc.SelfCalibration(baseline, d)
fringestot = ca.compute_fringes(q, doplot=False, theta=np.array([np.pi/25]))
_, quart_fp = sc.get_real_fp(fringestot[:, :, 0], quadrant=2)
quart_fp = np.flip(quart_fp, axis=1)

rc('figure', figsize=(16,7))
subplot(1,2,1)
imshow(np.nan_to_num(quart_fp[:,:] * mask), cmap='bwr', interpolation='Gaussian')
colorbar()